Домашнее задание: Regular Expressions
==
В рамках данного задания Вам необходимо работать с [данным новостным сайтом](http://news-rg.task-sss.krasilnikov.spb.ru/ad7b25e2374b423547e6783add6d7428/1).

**НЕ изменяйте и не удаляйте ячейки, написанные с помощью markdown, только ячейки с кодом**

Для выполнения заданий Вам необходимо заполнить написать код, который решает ту или иную задачу, и вставить его в ячейки.

Удачи!

### Задание №1 (**2 балла**)
Соберите информацию(*url новости, заголовок новости и ее тематика (тэг)*) о новостях со всех страниц сайта. Поместите эту информацию в датафрейм, со следующими колонками:
* `url` - ссылка (должна быть полной, т.е. начинаться с http или https) на новость (тип колонки - `str`).
* `title` - заголовок новости (тип колонки - `str`).
* `tag` - категория новости (новости/истории/шапито/...) (тип колонки - `str`, не должен содержать ковычек).

Поместите этот датафрейм в переменную `news_tag`.

**Примечание(!!!)**: Информацию о новостях нужно собрать со всех страниц сайта. Количество страниц с новостями может поменяться (даже после дедлайна). Поэтому Вам не следует использовать конкретное число страниц внутри Вашего кода. Все поля не должны содержать лишних пробельных символов и символов переноса строк (`"\n"`). Порядок новостей в датафрейме должен быть тем же, что и на сайте.


In [1]:
### news_date=...
import re
import requests as rq
import pandas as pd

my_patterns = ['(?s)<a class="b-link b-link_title"\n                                       href="(.*?)">',
              '">\n                                        (.*?) </a></h2>','<a href="" class="b-link b-link_tag">(.*?)</a>',
              ' src="(.*?)" width="740"','"published_time": "(.*?)",\n            ','<p>(.*?)</p>']


def url(txt):
    list_1 = re.findall(my_patterns[0],txt)
    for i in range(len(list_1)):
        list_1[i] = 'http://news-rg.task-sss.krasilnikov.spb.ru' + list_1[i]
    data_a = pd.DataFrame(list_1).rename(columns={0:'url'})
    return data_a

def title_tag(txt):
    data_b = pd.DataFrame({'title':re.findall(my_patterns[1],txt), 'tag':re.findall(my_patterns[2],txt)})
    return data_b

def page_df(txt):
    data_c = pd.concat([url(txt),title_tag(txt)],axis=1)
    return data_c
    
while True:
    status = rq.get('http://news-rg.task-sss.krasilnikov.spb.ru/ad7b25e2374b423547e6783add6d7428/1')
    if status.status_code == 200:
        break
news_tag = page_df(status.text)

i = 2
oops = 0

while True:
    while True:
        stat = rq.get('http://news-rg.task-sss.krasilnikov.spb.ru/ad7b25e2374b423547e6783add6d7428/'+str(i))
        if (stat.status_code == 404):
            oops = 1
            break
        if (stat.status_code == 200):
            break
    if oops != 1:
        data_2 = page_df(stat.text)
        news_tag = news_tag.append(data_2)
        i += 1
    else:
        break
    
news_tag = news_tag.reset_index(drop=True)

,url,title,tag
0,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Губернатор Подмосковья объявил о самоизоляции ...,новости
1,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Картину Бэнкси «Покажи мне Моне» продали на ау...,новости
2,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В больницах заканчиваются места для пациентов ...,новости
3,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Правда ли белорусских силовиков научились расп...,истории
4,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В Москве готовят новый карантин для пожилых лю...,новости
...,...,...,...
261,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Антиперспиранты и загар топлес вызывают рак гр...,партнерский материал
262,http://news-rg.task-sss.krasilnikov.spb.ru/new...,На сотрудницу красноярского МВД завели дело о ...,новости
263,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В 1666 году король Карл II разрешил рыбакам из...,новости
264,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"На побережье Бразилии нашли мертвого пингвина,...",новости


### Задание №2 (**2 балла**)
Для каждой новости со всех страниц сайта соберите информацию об изображении в тексте статьи и точное время ее публикации. Поместите данную информацию а датафрейм со следующими колонками:

* `url` - ссылка (должна быть полной, т.е. начинаться с http или https) на новость (тип колонки - `object`).
* `title` -  заголовок новости (тип колонки - `object`).
* `pic_url` - URL картинки в тексте статьи. На каждой из них есть логотип "Meduza" (тип колонки - `object`).
* `time` - точное время, когда была опубликована новость в формате `dd.mm.yyyy hh:mm` (тип колонки - `object`). 

Поместите этот датафрейм в переменную `news_pictures`.

**Примечание(!!!)**: Информацию о новостях нужно собрать со всех страниц сайта. Количество страниц с новостями может поменяться (даже после дедлайна). Поэтому Вам не следует использовать конкретное число страниц внутри Вашего кода. Все поля не должны содержать лишних пробельных символов и символов переноса строк (`"\n"`). 
Порядок новостей в датафрейме должен быть тем же, что и на сайте.

In [4]:
### news_pictures=...
urls = list(news_tag['url'])
articles_str = ' '

    
def pic_df(n):
    while True:
        status = rq.get(str(urls[n]))
        if status.status_code == 200:
            break
    data_d = pd.DataFrame({'pic_url':re.findall(my_patterns[3],status.text), 
                               'time':(re.findall(my_patterns[4],status.text))[0].replace('T',' ')})
    return data_d

def text_conc(txt):
    stroka = ''
    pl = re.findall(my_patterns[5],txt)
    for i in range(len(pl)):
        stroka += pl[i]
    return stroka

while True:
        status = rq.get(str(urls[0]))
        if status.status_code == 200:
            break
            
news_pictures = pd.DataFrame({'pic_url':re.findall(my_patterns[3],status.text), 
                               'time':(re.findall(my_patterns[4],status.text))[0].replace('T',' ')})
articles_str = text_conc(status.text)
articles_list = [articles_str]


for j in range(1,20):
    while True:
        status = rq.get(str(urls[int(j)]))
        if status.status_code == 200:
            break
    data_d = pd.DataFrame({'pic_url':re.findall(my_patterns[3],status.text), 
                               'time':(re.findall(my_patterns[4],status.text))[0].replace('T',' ')})
    news_pictures = news_pictures.append(data_2)
    articles_str += text_conc(status.text)
    x = text_conc(status.text)
    articles_list.append(x)
    
for k in range(20,len(urls)):
    data_2 = pic_df(k)
    news_pictures = news_pictures.append(data_2)


news_pictures = pd.concat([news_tag[['url','title']],news_pictures.reset_index(drop=True)],axis=1)

,url,title,pic_url,time
0,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Губернатор Подмосковья объявил о самоизоляции ...,https:/meduza.io//imgly/share/1604139672/news/...,2020-09-07 18:09:00
1,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Картину Бэнкси «Покажи мне Моне» продали на ау...,https:/meduza.io//imgly/share/1601310761/episo...,2020-09-07 18:09:00
2,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В больницах заканчиваются места для пациентов ...,https:/meduza.io//imgly/share/1601310761/episo...,2020-09-07 18:09:00
3,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Правда ли белорусских силовиков научились расп...,https:/meduza.io//imgly/share/1601310761/episo...,2020-09-07 18:09:00
4,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В Москве готовят новый карантин для пожилых лю...,https:/meduza.io//imgly/share/1601310761/episo...,2020-09-07 18:09:00
...,...,...,...,...
261,http://news-rg.task-sss.krasilnikov.spb.ru/new...,Антиперспиранты и загар топлес вызывают рак гр...,https:/meduza.io//imgly/share/1603811050/featu...,2020-09-07 18:09:00
262,http://news-rg.task-sss.krasilnikov.spb.ru/new...,На сотрудницу красноярского МВД завели дело о ...,https:/meduza.io//imgly/share/1603291404/news/...,2020-09-07 18:09:00
263,http://news-rg.task-sss.krasilnikov.spb.ru/new...,В 1666 году король Карл II разрешил рыбакам из...,https:/meduza.io//imgly/share/1602249913/featu...,2020-09-07 18:09:00
264,http://news-rg.task-sss.krasilnikov.spb.ru/new...,"На побережье Бразилии нашли мертвого пингвина,...",https:/meduza.io//imgly/share/1600343319/news/...,2020-09-07 18:09:00


### Задание №3 (**3 балла**)

Для выполнения следующего задания ознакомьтесь с библиотекой [pymystem3](https://pypi.org/project/pymystem3/) (Библиотеку, скорее всего, нужно будет установить. Для установки библиотек есть [официальная инструкция](https://packaging.python.org/tutorials/installing-packages/)).

Запишите в переменную `country_count`, сколько раз в новостях с первых двух(!) страниц сайта (тексте и заголовках) успомянались страны: Россия, Китай и США. Искать нужно по словам "Россия", "Российский", "Китай", "Китайский", "США", "Америка", "Американский". Упомянания должны считаться вне зависимости от педажей, регистра или рода прилагательного.

In [5]:
### country_count=...

import pymystem3 as py

titles = list(news_tag['title'])[0:20]
titles_list = str(titles[0])
for i in range(1,len(titles)):
    titles_list += str(titles[i])
from pymystem3 import Mystem
m = Mystem()
lemmas = m.lemmatize(articles_str+titles_list)
lemmas = ''.join(lemmas).lower()
pattern = "россия|российский|китай|китайский|сша|америка|американский"
result = re.findall(pattern, lemmas)
country_count = len(result)

92

### Задание №4 (**4 балла**)

В этом задании Вам необходимо расчитать метрику TF-IDF: метрика важности слова в новости или в любом другом документе (Программировать формулу не надо, достаточно воспользоваться [готовой реализацией](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html). Все параметры оставьте значениями по умолчанию. На выходе этот алгоритм выдает sparse-матрицы. Все необходимые функции для выполнения задание Вы можете найти [здесь](https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html)). Для каждой новости с первых двух(!) страниц сайта и для каждого слова в текстах этих новостей (без заголовков). Перед расчетом метрики необходимо: 
1) оставить в тексте только буквы и пробельные знаки 
2) привести все слова к нижнему регистру 
3) привести все слова к начальной форме с помощью библиотеки pymystem3

В переменную `important_words` запишите датафрейм со следующими колонками:
* `title` - заголовок новости (тип колонки `object`)
* `max_tfidf_word` - слово с максимальным значением TF-IDF (тип колонки `object`)
* `max_tfidf` - максимальное значение TF-IDF для каждой новости (тип колонки `float64`)
Новости должны быть в том порядке, в котором они представлены на сайте.

P.S. [Здесь](https://monkeylearn.com/blog/what-is-tf-idf/) можно прочитать про TF-IDF



In [7]:
### important_words=...
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

def tfidf(n):
    p_txt = articles_list[n]
    pattern = "[^а-яА-Яa-zA-Z ]"
    p_txt = re.sub(pattern, ' ', p_txt)
    p_txt = p_txt.lower()
    p_txt = ' '.join(m.lemmatize(p_txt))
    if p_txt == '':
        data_i = pd.DataFrame({'max_tfidf_word':[np.nan],'max_tfidf':[0.000]})
    else:
        vectorizer = TfidfVectorizer()
        X = vectorizer.fit_transform([p_txt])
        X = X.todense()
        voc = vectorizer.get_feature_names()
        data_i = pd.DataFrame({'max_tfidf_word':[voc[X.argmax()]],'max_tfidf':[np.max(X)]})
    return data_i

tf_idf = tfidf(0)

for i in range(1,20):
    tf_idf = tf_idf.append(tfidf(i))

important_words = pd.concat([news_tag[['title']].head(20),tf_idf.reset_index(drop=True)],axis=1)

,title,max_tfidf_word,max_tfidf
0,Губернатор Подмосковья объявил о самоизоляции ...,человек,0.395628
1,Картину Бэнкси «Покажи мне Моне» продали на ау...,на,0.339151
2,В больницах заканчиваются места для пациентов ...,на,0.364176
3,Правда ли белорусских силовиков научились расп...,на,0.366754
4,В Москве готовят новый карантин для пожилых лю...,москва,0.346873
5,«Иo side effects whatsoeveя». Актер Эдриан Бро...,вакцина,0.342791
6,Как крупный наркоторговец годами выдавал себя ...,ковальчук,0.463896
7,Миллиардер Юрий Мильнер профинансирует поиски ...,на,0.432512
8,Сбербанк и Mail.ru Group станут совладельцами ...,сервис,0.427482
9,ФСБ заявила о задержании участников «закрытого...,на,0.353103
